## Search Books by title, author or genre

In [32]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.schema.runnable.base import Runnable
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from typing import Type, Any, Optional, List
import json
import sqlite3 
from pydantic import BaseModel, Field
import ast
import openai


llm = ChatOpenAI(api_key= 'api_key', model='gpt-4o-mini')


In [33]:
class PromptTemplate(BaseModel):
    """Defines templates for system and human messages used in a conversation."""

    system_template: str = Field(
        description="Template for the system message in the conversation"
    )
    human_template: str = Field(
        description="Template for the human message in the conversation"
    )

def generate_prompt_templates(
    prompt_template: PromptTemplate
) -> ChatPromptTemplate:
    """Generate a chat prompt template based on given templates and memory setting.

    Args:
        prompt_template: An instance of PromptTemplate containing system and human templates.

    Returns:
        A configured ChatPromptTemplate with specified message structure.
    """

    # Create prompt template without chat history
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate.from_template(
                prompt_template.system_template
            ),
            HumanMessagePromptTemplate.from_template(
                prompt_template.human_template
            ),
        ]
    )

    return prompt

In [34]:
class WhichInput(BaseModel):
    which_input: str
    name: str


class ExtractInput(Runnable):
    def __init__(self, llm):
        super().__init__()

        self.llm = llm

        prompt_template = PromptTemplate(
            system_template=""" 
            You are a part of a book searching system team. 
            Your task is to analyze user queries and identify mentioned books, authors, or genres.
            If the user provides an author name, return 'author' and its name, if the user provides a genre, return 'genre' and its name, and if the user provides title, return 'title' and its name.
            Here is the user input:
            {user_input}

            {format_instructions}
            """,
            human_template="user input: {user_input}",
        )

        self.prompt = generate_prompt_templates(prompt_template)
        self.output_parser = PydanticOutputParser(pydantic_object=WhichInput)
        self.format_instructions = self.output_parser.get_format_instructions()

        self.chain = self.prompt | self.llm | self.output_parser


    def invoke(self, inputs):
        result = self.chain.invoke(
            {
                "user_input": inputs["user_input"],
                "format_instructions": self.format_instructions,
            })

        return result

In [ ]:
class SearchBookInput(BaseModel):
    search_input: str

class SearchBookTool(BaseTool):
    name: str = "SearchBookTool"
    description: str = "Search for a book based on title, author, or genre."
    args_schema: Type[BaseModel] = SearchBookInput
    return_direct: bool = True

    def _run(self, search_input: str) -> str:
        llm = ChatOpenAI(api_key= 'api_key', model='gpt-4o-mini')

        # Extract the type of search (book, author, or genre)
        u_input = ExtractInput(llm).invoke({"user_input": search_input})

        # Debugging: Check the parsed input
        #print(f"Parsed input: {u_input}")
        #print(f"Which input: {u_input.which_input}")
        #print(f"Input name: {u_input.name}")

        con = sqlite3.connect("db path")
        cursor = con.cursor()

        # Genre Search
        if u_input.which_input == 'genre':
            genre_name = u_input.name
            #print(f"Searching for genre: '{genre_name}'")  # Debugging

            cursor.execute(f"SELECT genre_id FROM genres WHERE LOWER(genre) = LOWER(?)", (genre_name,))
            genre_results = cursor.fetchall()
            #print(f"Genre results: {genre_results}")  # Debugging

            if not genre_results:
                return f"Genre '{genre_name}' not found in the database."

            genre_id = genre_results[0][0]

            query = """
            SELECT 
                b.book_id,
                b.title,
                GROUP_CONCAT(a.author_name, ', ') AS authors
            FROM books b
            INNER JOIN books_genres bg ON b.book_id = bg.book_id
            INNER JOIN authors_books ab ON b.book_id = ab.book_id
            INNER JOIN authors a ON ab.author_id = a.author_id
            WHERE bg.genre_id = ? 
            LIMIT 10;
            """

            cursor.execute(query, (genre_id,))
            books_by_genre = cursor.fetchall()
            #print(f"Books found: {books_by_genre}")  # Debugging

            if not books_by_genre:
                return f"No books found for the genre '{genre_name}'."

            result = "\n".join([f"{i+1}. {book[1]} by {book[2]}" for i, book in enumerate(books_by_genre)])
            return f"Books in the genre '{genre_name}':\n{result}"

        # Author Search
        if u_input.which_input == 'author':
            author_name = u_input.name.strip()
            #print(f"Searching for author: '{author_name}'")  # Debugging

            cursor.execute(f"SELECT author_id FROM authors WHERE LOWER(author_name) = LOWER(?)", (author_name,))
            author_results = cursor.fetchall()
            #print(f"Author results: {author_results}")  # Debugging

            if not author_results:
                return f"Author '{author_name}' not found in the database."

            author_id = author_results[0][0]

            query = """
            SELECT 
                b.book_id, 
                b.title   
            FROM books b
            INNER JOIN authors_books ab ON b.book_id = ab.book_id
            INNER JOIN authors a ON ab.author_id = a.author_id
            WHERE a.author_id = ?
            LIMIT 5;
            """

            cursor.execute(query, (author_id,))
            books_by_author = cursor.fetchall()
            #print(f"Books found: {books_by_author}")  # Debugging

            if not books_by_author:
                return f"No books found by the author '{author_name}'."

            result = "\n".join([f"{i+1}. {book[1]}" for i, book in enumerate(books_by_author)])
            return f"Books by '{author_name}':\n{result}"

        # Title Search
        if u_input.which_input == 'title':
            title = u_input.name.strip()  # Removes leading/trailing spaces
            #print(f"Searching for books with title: '{title}'")  # Debugging

            query = """
            SELECT 
                b.book_id,
                b.title,
                GROUP_CONCAT(a.author_name, ', ') AS authors
            FROM books b
            INNER JOIN authors_books ab ON b.book_id = ab.book_id
            INNER JOIN authors a ON ab.author_id = a.author_id
            WHERE LOWER(b.title) LIKE LOWER(?)
            GROUP BY b.book_id
            LIMIT 5;
            """

            cursor.execute(query, ('%' + title + '%',))
            books_by_title = cursor.fetchall()
            #print(f"Books found: {books_by_title}")  # Debugging

            if not books_by_title:
                return f"No books found with the title '{title}'."
            result = "\n".join([f"{i+1}. {book[1]} by {book[2]}" for i, book in enumerate(books_by_title)])
            return f"Books matching the title '{title}':\n{result}"

        con.close()


In [38]:
search_input = SearchBookTool()

print(search_input.invoke({'search_input': 'Search books by author Jack Williamson'}))
print(search_input.invoke({'search_input': 'Search books in the genre History'}))
print(search_input.invoke({'search_input': 'Search for books titled Harry Potter'}))

Books by 'Jack Williamson':
1. The Starchild Trilogy
2. The Singers of Time
Books in the genre 'History':
1. America at 1750: A Social Portrait by Richard Hofstadter, Abraham Lincoln, Thomas Paine, Howard Zinn, Will Durant, Ariel Durant, David Remnick, Howard Zinn, John Matthews, Richard Pipes, John Keegan, Tom Brokaw, Mary Ellen Snodgrass, Henry David Thoreau, Tom Brokaw, Stephen E. Ambrose, Various Authors, James M. McPherson, George Henry Davis `86 Professor of American History James M McPherson, William James Cooper (Jr.), Andrew Murray, Farley Mowat, John Reed, General Press, McGraw-Hill Education, Walter Mosley, Daniel Cohen, Gary W. Gallagher, Jim Bishop, Arthur Koestler, Margaret MacMillan, Mari Sandoz, Henry David Thoreau, Martin Gilbert, John Grant, James M. Lynch, Ronald H. Bailey, Ted Spiegel, Orlando Figes, Gary W. Gallagher, David Harris, Time-Life Books, Martin Gilbert, Stephen Hunter, John Bainbridge, Frances Gies, Joseph Gies, Karl Marx, Lewis Mumford, David Barton, Vi

In [ ]:
#con = sqlite3.connect("path to db")
#cursor = con.cursor()
#cursor.execute("SELECT author_name FROM authors LIMIT 5;")
#books = cursor.fetchall()
#print(books)
